# Homework 2: Word Similarity

Student Name:Wai Yan Wong

Student ID:892083

## General info

<b>Due date</b>: Thursday, 18 Apr 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dateset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/">Similarity-353</a>. You need to first obtain this dataset, which is available on LMS. The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.

For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *filtered_gold_standard*.t

(1 mark)

In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
###

from collections import Counter
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

# Code below from WSTA_N10_distributional_semantics
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
# End of copied code
        
processed_docs = []
all_token_freq = Counter()
term_doc_freq = Counter()
for paragraph in brown.paras():
    para = []
    for sentence in paragraph:
        for token in sentence:
            if token.isalpha():
                word = lemmatize(token.lower())
                if word not in para:
                    term_doc_freq[word] += 1
                para.append(word)
                all_token_freq[word] += 1
    processed_docs += [para]   

import csv
with open('xid-59719076_2.tab', newline='') as set1:
    reader = csv.reader(set1, delimiter='\t')
    for line in reader:
        if term_doc_freq[line[0]] >= 8 and term_doc_freq[line[1]] >= 8:
            filtered_gold_standard[(line[0],line[1])] = float(line[2])

###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Error loading brown: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>


94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('tennis', 'racket'): 7.56, ('law', 'lawyer'): 8.38, ('movie', 'star'): 7.38, ('movie', 'critic'): 6.73, ('movie', 'theater'): 7.92, ('space', 'chemistry'): 4.88, ('alcohol', 'chemistry'): 

<b>For your testing: </b>

In [2]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [3]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

(1 mark)

In [4]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###

primary_sense = {}

def single_primary_sense(word):
    synsets = wordnet.synsets(word)
    if len(synsets) > 0:
        if len(synsets) == 1:
            primary_sense[word] = synsets[0]
            return True
        else:
            sense_count = [0 for x in range(0,len(synsets))]
            for i, synset in enumerate(synsets):
                for lemma in synset.lemmas():
                    if(lemma.name().lower() == word):
                        sense_count[i] += lemma.count()
            max_count = max(sense_count)
            index = sense_count.index(max_count)
            sense_count.remove(max_count)
            second_count = max(sense_count)               
            if max_count >= 4*second_count:
                primary_sense[word] = synsets[index]
                return True
    return False

    
def primary_sense_is_noun(word):
    sense = primary_sense[word]
    if sense.name().split(".")[1] == "n":
        return True
    return False

for wordpair in filtered_gold_standard:
    if single_primary_sense(wordpair[0]) and single_primary_sense(wordpair[1]):
        if primary_sense_is_noun(wordpair[0]) and primary_sense_is_noun(wordpair[1]):
            final_gold_standard[wordpair] = filtered_gold_standard[wordpair]
            

###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [5]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [6]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA (3 marks)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

(1 mark)

In [15]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

semcor_ic = wordnet_ic.ic('ic-brown.dat')
for wordpair in final_gold_standard:
    sense0 = primary_sense[wordpair[0]]
    sense1 = primary_sense[wordpair[1]]
    lin_similarities[wordpair] = sense0.lin_similarity(sense1,semcor_ic)


###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Error loading wordnet_ic: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>


{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psychology', 'health'): 0.06004979886905243, ('psychology', 'science'): 0.8474590505736942, ('planet', 'moon'): 0.7808882364067532, ('planet', 'galaxy'

<b>For your testing:</b>

In [8]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NMPI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

(1 mark)

In [9]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math

num_co_occur = Counter()
for wordpair in final_gold_standard:
    word1 = wordpair[0]
    word2 = wordpair[1]    
    for doc in processed_docs:
        word1_exist = False
        word2_exist = False
        for token in doc:
            if token == word1:
                word1_exist = True
            if token == word2:
                word2_exist = True
        if word1_exist == True and word2_exist == True:
            num_co_occur[wordpair] += 1

total_word_count = 0.0
for word in term_doc_freq:
    total_word_count += term_doc_freq[word]

for wordpair in final_gold_standard:
    if num_co_occur[wordpair] == 0:
        NPMI_similarities[wordpair] = -1
    else:
        pxy = num_co_occur[wordpair]/total_word_count
        px = term_doc_freq[wordpair[0]]/total_word_count
        py = term_doc_freq[wordpair[1]]/total_word_count
        NPMI_similarities[wordpair] = (math.log(px*py,2)/math.log(pxy,2)) -1 

###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.6531272737497535, ('professor', 'doctor'): -1, ('student', 'professor'): 0.535495995400926, ('stock', 'egg'): 0.3685128922213745, ('money', 'cash'): 0.44493834723517267, ('king', 'queen'): 0.4181407297139579, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.7161994042283006, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6246376972254173, ('baby', 'mother'): 0.5149353890388502, ('car', 'automobile'): 0.5430334549802613, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.586151062984337, ('brother', 'monk'): 0.4299318525693172, ('journey', 'car'): -1, ('coast', 'hill'): 0.3393102883484622, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.45787396913892864, ('psychology', 'doctor'): 0.46132815837264096, ('psychology', 'mind'): 0.44616660434351796, ('psychology', 'health'): -1, ('psychology', 'science'): 0.5908740096534866, ('planet', 'moon'): 0.6580006162323315, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [10]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

(1 mark)

In [11]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###

from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = 1
    return BOW

texts = []
for doc in processed_docs:
    texts.append(get_BOW(doc))

vectorizer = DictVectorizer()
brown_matrix = vectorizer.fit_transform(texts).T

svd = TruncatedSVD(n_components=500)
brown_matrix_lowrank = svd.fit_transform(brown_matrix)


word1_vector = []
word2_vector = []
for wordpair in final_gold_standard:
    for i, feature in enumerate(vectorizer.get_feature_names()):  
        if wordpair[0] == feature:
            word1_vector = brown_matrix_lowrank[i]
        if wordpair[1] == feature:
            word2_vector = brown_matrix_lowrank[i]
    cos_dis = 1 - cosine(word1_vector,word2_vector)
    LSA_similarities[wordpair] = cos_dis
    

###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.31590254861870015, ('professor', 'doctor'): 0.03452670758616394, ('student', 'professor'): 0.26765198811183355, ('stock', 'egg'): 0.1136383255949086, ('money', 'cash'): 0.12811109227319317, ('king', 'queen'): 0.13818047599580918, ('bishop', 'rabbi'): 0.027046229779748132, ('football', 'basketball'): 0.26828195561055357, ('football', 'tennis'): 0.12652549759788, ('alcohol', 'chemistry'): 0.1113468213220149, ('baby', 'mother'): 0.3729563740140729, ('car', 'automobile'): 0.3569719233195643, ('journey', 'voyage'): 0.08788698272143569, ('coast', 'shore'): 0.4260475397124004, ('brother', 'monk'): 0.08399865729862444, ('journey', 'car'): -0.018667941068820637, ('coast', 'hill'): 0.21062068421905977, ('forest', 'graveyard'): 0.039519093165331975, ('monk', 'slave'): -0.024454703254617183, ('coast', 'forest'): 0.14746024813868697, ('psychology', 'doctor'): 0.1352949399138913, ('psychology', 'mind'): 0.12037109930744139, ('psychology', 'health'): 0.03828232904118567, ('psy

<b>For your testing:</b>

In [12]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard (1 mark)


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(1 mark)


In [13]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

original_result = list(range(len(final_gold_standard)))
lin_result = original_result.copy()
NPMI_result = original_result.copy()
LSA_result = original_result.copy()
i = 0
for i,wordpair in enumerate(final_gold_standard):
    original_result[i] = final_gold_standard[wordpair]
    lin_result[i] = lin_similarities[wordpair]
    NPMI_result[i] = NPMI_similarities[wordpair]
    LSA_result[i] = LSA_similarities[wordpair]

def pearsonr_result(similarities):
    r,p = pearsonr(original_result,similarities)
    return r

pearson_correlations['lin'] = pearsonr_result(lin_result)
pearson_correlations['NPMI'] = pearsonr_result(NPMI_result)
pearson_correlations['LSA'] = pearsonr_result(LSA_result)

###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.5301489978447533, 'NPMI': 0.18799888869410047, 'LSA': 0.42909498292757686}


<b>For your testing:</b>

In [14]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## Challenge yourself: Improving the correlation (This part will NOT be marked)

You can try to derive a similarity score from word2vec vectors, using the Gensim interface, and compare it with the similarity scores you've created and the gold standard. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes).

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.